In [1]:
import pandas as pd
import csv
import jieba
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import collections

In [2]:
file_path = ".csv"

In [3]:
def read_csv_as_list(file_path):
    data = []
    with open(file_path, 'r', newline='') as file:
        csv_reader = csv.reader(file)
        first_row = True
        for row in csv_reader:
            if first_row:  
                first_row = False
                continue
            data.append(row)
    return data

In [4]:
csv_data = read_csv_as_list(file_path)

In [5]:
for idex in csv_data:
#     if len(idex[0])>500:
#         print(len(idex[0]))
#         print(idex[0])
    if len(idex[1])>1000:
        print(len(idex[1]))
        print(idex[1])

1286
DRAM BITPart of the memory is tested in Boot Sequence 1, and part of the memory is tested during Boot Sequence 2.  During Boot Sequence 1, ? KB of DRAM is tested before the Bootstrap is copied to this area.  During Boot Sequence 2, the  remaining memory is tested in ? KB blocks. The DRAM BIT first writes the address of each word in a block to that word in the block.   Then the test flushes the cache and reads back the entire block.  If a single-bit error occurs on the read-back, the location is read again.  If another single-bit error occurs, the test assumes a hard error and marks the whole block bad.  If a multiple-bit error occurs, the whole block is marked bad.  If the value read back does not match the address, the whole block is marked bad.  This same process is repeated with the ones complement of the address. The Bootstrap stores the results of the DRAM BIT in EEPROM as a series of x consecutive y-bit words.  Each bit in a result word represents the result of testing a ? K

In [6]:
text = "Command and Control CSCThe Command and Control (CCM) CSC is a Level 2 reuse component from the INSTRUMENT Y project.  The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.Major data structures include:* A Static Data table that keeps track of the operational state.  Data included in this table includes housekeeping production rates, CCM specific flags, and the number of commands executed.* A Command Queue into which commands are placed when they arrive, via interrupt, from the ICU or the SCU.* An Error/Event Queue which accumulates error and event codes which are reported by the DPU FSW.  These error and event codes are removed from the queue and placed into a telemetry packet at a given interval and included in DPU housekeeping data.DPUSDS5.12.1.1.1Flight Software InitializationThe Command and Control CSC is initialized by spawning the CCM Control Task, ccmCtrlTask(), from the operating system startup task, usrRoot().  After the task is spawned it calls a function, ccmInit(), which creates the error/event queue, instantiates needed semaphores and installs various ISRs.  Finally, it spawns the remaining tasks which comprise the DPU FSW. When the CCM Control Task starts up, it reads DPU configuration startup defaults from the SYSTEM_CONFIG_AREA in EEPROM.  If the SYSTEM_CONFIG_AREA checksum is bad, hard-coded defaults are used.  The CCM Control Task initializes the remaining CSCs by calling the applicable initialization function, or if the CSC has an associated task, by spawning the task using the VxWorks function, taskSpawn().  When the CCM Control Task initializes a CSC, it passes the startup defaults read from EEPROM as parameters to the task initialization function. In addition to its task initialization activities, ccmInit(), also initializes the command length verification table by calling ccmCmdLengthInit().  The command processor uses the created table to verify expected command lengths for newly received commands."

In [7]:
stop_words = set()
with open("E:/jupyterCode/data/stop_words/stopwords.txt", "r", encoding="utf-8") as f:
    for word in f:
        stop_words.add(word.strip())

# TextRank

In [9]:
import nltk
import numpy as np
import networkx as nx
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')  

def text_rank_summary(document, top_n=5):
    sentences = sent_tokenize(document)  

    if len(sentences) < top_n:
        return sentences  

    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    
    sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)  

    
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    
    summary = [sent for _, sent in ranked_sentences[:top_n]]
    
    return " ".join(summary)  


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
summary = text_rank_summary(text, top_n=3)
print(summary)

These error and event codes are removed from the queue and placed into a telemetry packet at a given interval and included in DPU housekeeping data.DPUSDS5.12.1.1.1Flight Software InitializationThe Command and Control CSC is initialized by spawning the CCM Control Task, ccmCtrlTask(), from the operating system startup task, usrRoot(). The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.Major data structures include:* A Static Data table that keeps track of the operational state. When the CCM Control Task initializes a CSC, it passes the startup defaults re

In [14]:
text

'Command and Control CSCThe Command and Control (CCM) CSC is a Level 2 reuse component from the INSTRUMENT Y project.  The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.Major data structures include:* A Static Data table that keeps track of the operational state.  Data included in this table includes housekeeping production rates, CCM specific flags, and the number of commands executed.* A Command Queue into which commands are placed when they arrive, via interrupt, from the ICU or the SCU.* An Error/Event Queue which accumulates error and event codes wh

In [12]:
len(text)

2392

In [13]:
len(summary)

1065

# Liu M, Wang L, Nie L. Weibo-oriented Chinese news summarization via multi-feature combination[C]//Natural Language Processing and Chinese Computing: 4th CCF Conference, NLPCC 2015, Nanchang, China, October 9-13, 2015, Proceedings 4. Springer International Publishing, 2015: 581-589.

In [54]:
def split_sentence(text, punctuation_list='.!?。！？'):

    sentence_set = []
    inx_position = 0         
    char_position = 0        
    for char in text:
        char_position += 1
        if char in punctuation_list:
            next_char = list(text[inx_position:char_position+1]).pop()
            if next_char not in punctuation_list:
                sentence_set.append(text[inx_position:char_position])
                inx_position = char_position
    if inx_position < len(text):
        sentence_set.append(text[inx_position:])
 
    sentence_with_index = {i:sent for i,sent in enumerate(sentence_set)} 
    return sentence_set,sentence_with_index

In [33]:
def get_tfidf_matrix(sentence_set,stop_word):
    corpus = []
    for sent in sentence_set:
        sent_cut = jieba.cut(sent)
        sent_list = [word for word in sent_cut if word not in stop_word]
        sent_str = ' '.join(sent_list)
        corpus.append(sent_str)
 
    vectorizer=CountVectorizer()
    transformer=TfidfTransformer()
    tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus))
    # word=vectorizer.get_feature_names()
    tfidf_matrix=tfidf.toarray()
    return tfidf_matrix

In [34]:
def get_sentence_with_words_weight(tfidf_matrix):
    sentence_with_words_weight = {}
    for i in range(len(tfidf_matrix)):
        sentence_with_words_weight[i] = np.sum(tfidf_matrix[i])
 
    max_weight = max(sentence_with_words_weight.values()) #归一化
    min_weight = min(sentence_with_words_weight.values())
    for key in sentence_with_words_weight.keys():
        x = sentence_with_words_weight[key]
        sentence_with_words_weight[key] = (x-min_weight)/(max_weight-min_weight)
 
    return sentence_with_words_weight

In [35]:
def get_sentence_with_position_weight(sentence_set):
    sentence_with_position_weight = {}
    total_sent = len(sentence_set)
    for i in range(total_sent):
        sentence_with_position_weight[i] = (total_sent - i) / total_sent
    return sentence_with_position_weight

In [36]:
def similarity(sent1,sent2):

    return np.sum(sent1 * sent2) / (1e-6+(np.sqrt(np.sum(sent1 * sent1)) *\
                                    np.sqrt(np.sum(sent2 * sent2))))
 
def get_similarity_weight(tfidf_matrix):
    sentence_score = collections.defaultdict(lambda :0.)
    for i in range(len(tfidf_matrix)):
        score_i = 0.
        for j in range(len(tfidf_matrix)):
            score_i += similarity(tfidf_matrix[i],tfidf_matrix[j])
        sentence_score[i] = score_i
 
    max_score = max(sentence_score.values()) 
    min_score = min(sentence_score.values())
    for key in sentence_score.keys():
        x = sentence_score[key]
        sentence_score[key] = (x-min_score)/(max_score-min_score)
 
    return sentence_score

In [37]:
def ranking_base_on_weigth(sentence_with_words_weight,
                            sentence_with_position_weight,
                            sentence_score, feature_weight = [1,1,1]):
    sentence_weight = collections.defaultdict(lambda :0.)
    for sent in sentence_score.keys():
        sentence_weight[sent] = feature_weight[0]*sentence_with_words_weight[sent] +\
                                feature_weight[1]*sentence_with_position_weight[sent] +\
                                feature_weight[2]*sentence_score[sent]
 
    sort_sent_weight = sorted(sentence_weight.items(),key=lambda d: d[1], reverse=True)
    return sort_sent_weight

In [59]:
def get_summarization(sentence_with_index,sort_sent_weight,topK_ratio =0.25):
    topK = int(len(sort_sent_weight)*topK_ratio)
    summarization_sent = sorted([sent[0] for sent in sort_sent_weight[:topK]])
    
    summarization = []
    for i in summarization_sent:
        summarization.append(sentence_with_index[i])
 
    summary = ''.join(summarization)
    return summary

In [60]:
sentence_set, sentence_with_index = split_sentence(text)
tfidf_matrix = get_tfidf_matrix(sentence_set, stop_words)
sentence_with_words_weight = get_sentence_with_words_weight(tfidf_matrix)
sentence_with_position_weight = get_sentence_with_position_weight(sentence_set)
sentence_score = get_similarity_weight(tfidf_matrix)
sort_sent_weight = ranking_base_on_weigth(sentence_with_words_weight, sentence_with_position_weight, sentence_score)
summary = get_summarization(sentence_with_index, sort_sent_weight)

print("Summary:\n", summary)

Summary:
 Command and Control CSCThe Command and Control (CCM) CSC is a Level 2 reuse component from the INSTRUMENT Y project.  The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.  Data included in this table includes housekeeping production rates, CCM specific flags, and the number of commands executed.  These error and event codes are removed from the queue and placed into a telemetry packet at a given interval and included in DPU housekeeping data.1Flight Software InitializationThe Command and Control CSC is initialized by spawning the CCM Control Task

In [58]:
len(summary)

884

In [53]:
text

'Command and Control CSCThe Command and Control (CCM) CSC is a Level 2 reuse component from the INSTRUMENT Y project.  The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.Major data structures include:* A Static Data table that keeps track of the operational state.  Data included in this table includes housekeeping production rates, CCM specific flags, and the number of commands executed.* A Command Queue into which commands are placed when they arrive, via interrupt, from the ICU or the SCU.* An Error/Event Queue which accumulates error and event codes wh

In [48]:
len(text)

2392

# LLM

In [65]:
from transformers import pipeline

In [ ]:
# "jordiclive/flan-t5-3b-summarizer",

In [70]:
summarizer =pipeline("summarization","philschmid/bart-large-cnn-samsum")

In [71]:
summarizer(text,min_length=5,max_length=500)

[{'summary_text': 'Command and Control CSC is a Level 2 reuse component from the INSTRUMENT Y project. It includes a Control Task, ccmCtrlTask, which initializes the DPU FSW and spawns other tasks at bootup. It also includes a Command Dispatch Task, which receives and dispatches real-time commands received from the SCU or the ICU. Data structures include a Static Data table and an Error/Event Queue.'}]

In [72]:
summary

'Command and Control CSCThe Command and Control (CCM) CSC is a Level 2 reuse component from the INSTRUMENT Y project.  The Command and Control CSC includes the following components:* A Control Task, ccmCtrlTask(), which initializes the DPU FSW and spawns other tasks at bootup, schedules the production of DPU housekeeping data packets, monitors the execution of other tasks, and schedules the execution of other periodic tasks such as the heartbeat message and the watchdog strobe; and* A Command Dispatch Task, ccmCmdTask(), which receives and dispatches real-time commands received from the SCU or the ICU.  Data included in this table includes housekeeping production rates, CCM specific flags, and the number of commands executed.  These error and event codes are removed from the queue and placed into a telemetry packet at a given interval and included in DPU housekeeping data.1Flight Software InitializationThe Command and Control CSC is initialized by spawning the CCM Control Task, ccmCtrl